In [1]:
import random
import json
import math
import functools
import operator
from websocket_server import WebsocketServer
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class ModelState: 
    def __init__(self, x, y, theta):
        self.x = x
        self.y = y
        self.theta = theta
        
    def to_world_coords(self, px, py):
        xt = self.x + (math.cos(self.theta) * px) - (math.sin(self.theta) * py)
        yt = self.y + (math.sin(self.theta) * px) + (math.cos(self.theta) * py)
        return [xt, yt]        
    
    def __repr__(self):
        return "[x=%.3f, y=%.3f, theta=%.3f]" % (self.x, self.y, self.theta)

class CtrvMotionModel: 
    def __init__(self, stddev):
        self.stddev = stddev
        
    def init(self, x, y, theta):
        x, y, theta = self.gen_noise(x, y, theta)
        return ModelState(x, y, theta)
        
    def predict(self, prev_state, delta_t, vel, yawrate):
        theta = prev_state.theta + yawrate*delta_t
        if math.fabs(yawrate) > 1e-4:
            x = prev_state.x + vel/yawrate * (math.sin(theta) - math.sin(prev_state.theta))
            y = prev_state.y + vel/yawrate * (-math.cos(theta) + math.cos(prev_state.theta))
        else:
            x = prev_state.x + vel * math.cos(prev_state.theta) * delta_t
            y = prev_state.y + vel * math.sin(prev_state.theta) * delta_t
            
        x, y, theta = self.gen_noise(x, y, theta)
        return ModelState(x, y, theta)
    
    def gen_noise(self, x, y, theta):
        return [random.gauss(mean, std) for mean, std in zip([x, y, theta], self.stddev)]


In [3]:
class Landmark:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return "Landmark[%.3f, %.3f]" % (self.x, self.y)
    
    def distance_to(self, other):
        return math.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)
    
    
class MapData:
    def __init__(self, landmarks):
        self.landmarks = landmarks
        
    def find_nearest(self, observation):
        measured_dist = [(lm, lm.distance_to(observation)) for lm in self.landmarks]
        return min(measured_dist, key=(lambda x: x[1]))

def read_map(filename):
    def read_landmark(line):
        values = line.split()
        x = float(values[0])
        y = float(values[1])
        return Landmark(x, y)
    
    with open(filename) as f:
        return MapData([read_landmark(line) for line in f.readlines()])
            

        
        

In [4]:
def gauss_2d(x, mean, stddev):
    scale_factor = 1/(2*math.pi*stddev[0]*stddev[1])
    xx = (x[0] - mean[0])**2/(2*stddev[0]**2)
    yy = (x[1] - mean[1])**2/(2*stddev[1]**2)
    return scale_factor * math.exp(-(xx + yy))

def product(values):
    return functools.reduce(operator.mul, values, 1.0)

class ObservationModel:
    def __init__(self, map_data, stddev_obs):
        self.stddev = stddev_obs
        self.map_data = map_data
        
    def map_observations_to_world(self, state, observations):
        def to_map_coords(o):
            xmap, ymap = state.to_world_coords(o.x, o.y)
            return Landmark(xmap, ymap)
        
        return [to_map_coords(o) for o in observations]
    
    def associate_with_nearest_landmark(self, observations):
        def assoc(obs):
            nearest, _ = self.map_data.find_nearest(obs)
            return (obs, nearest)
        return [assoc(obs) for obs in observations]
    
    def calculate_probs(self, assocs):
        def prob(obs, lmark):
            return gauss_2d([obs.x, obs.y], [lmark.x, lmark.y], self.stddev)
        
        return [prob(obs, lmark) for (obs, lmark) in assocs]
        
        
        
    def calculate_weight(self, state, observations):
        world_obs = self.map_observations_to_world(state, observations)
        assocs = self.associate_with_nearest_landmark(world_obs)
        return product(self.calculate_probs(assocs))


In [5]:
class Particle:
    def __init__(self, iden, state):
        self.iden = iden
        self.state = state
        self.weight = 0.0
        self.associations = []
        self.sense_x = []
        self.sense_y = []
        
    def predict(self, delta_t, velocity, yawrate, sigma):
        return self


class ParticleFilter:
    n_particles = 10
    
    def __init__(self, motion_model, observation_model):
        self.particles = []
        self.motion_model = motion_model
        self.observation_model = observation_model
        self.is_initialized = False
        
        
    def init(self, sense_x, sense_y, sense_theta):
        for i in range(self.n_particles):
            init_state = self.motion_model.init(sense_x, sense_y, sense_theta)
            self.particles.append(Particle(i, init_state))
            
        self.is_initialized = True
    
    
    def predict(self, delta_t, velocity, yawrate):
        def _predict(particle):
            new_state = motion_model.predict(particle.state, delta_t, velocity, yawrate)
            return Particle(particle.iden, new_state)
        
        self.particles = [_predict(p) for p in self.particles]
    
    def update_weights(self, sensor_range, sigma_obs, observations):
        for p in self.particles:
            p.weight = self.observation_model.calculate_weight(p.state, observations)
        
    def find_best(self):
        best_weight = -1.0
        best_part = None
        
        for p in self.particles:
            if p.weight > best_weight:
                best_weight = p.weight
                best_part = p
        
        return p

In [6]:
class SimulatorResponder:
    valid_prefix = '42'
    sigma_pos = [0.3, 0.3, 0.01]
    delta_t = 0.1
    sigma_landmark = [0.3, 0.3]
    sensor_range = 50
    
    def __init__(self, particle_filter):
        self.pf = particle_filter
    
    def on_connect(self, client, server):
        print("Client connected")
        
    def is_valid_message(self, message):
        return message.startswith(self.valid_prefix)
    
    def parse_data(self, message):
        data_start = len(self.valid_prefix)
        data = json.loads(message[data_start:])
        if data[0] == 'telemetry':
            return data[1]
        else:
            return None
        
    def parse_landmarks(self, obs_x, obs_y):
        xx = [float(x) for x in obs_x.split()]
        yy = [float(y) for y in obs_y.split()]
        return [Landmark(x, y) for (x, y) in zip(xx, yy)]
        
    def best_particle(self):
        best = self.pf.find_best()
        return {
          'best_particle_associations': '',
          'best_particle_sense_x': '',
          'best_particle_sense_y': '',
          'best_particle_theta': best.state.theta,
          'best_particle_x': best.state.x,
          'best_particle_y': best.state.y
        }
        
    def respond(self, client, event, data):
        response = self.valid_prefix + json.dumps([event, data])
        client['handler'].send_message(response)
        
    def on_message(self, client, server, message):
        if not self.is_valid_message(message):
            return
        
        data = self.parse_data(message)
        if data is None: 
            self.respond(client, 'manual', {})
            return
        
        if not self.pf.is_initialized:
            sense_x = float(data['sense_x'])
            sense_y = float(data['sense_y'])
            sense_theta = float(data['sense_theta'])
            self.pf.init(sense_x, sense_y, sense_theta)
        else:
            prev_velocity = float(data['previous_velocity'])
            prev_yawrate = float(data['previous_yawrate'])
            self.pf.predict(self.delta_t, prev_velocity, prev_yawrate)
            
            observations = self.parse_landmarks(data['sense_observations_x'], data['sense_observations_y'])
            self.pf.update_weights(self.sensor_range, self.sigma_landmark, observations)

        self.respond(client, 'best_particle', self.best_particle())
        
    

In [ ]:
motion_model = CtrvMotionModel([0.3, 0.3, 0.01])
observation_model = ObservationModel(read_map('./data/map_data.txt'), [0.3, 0.3])
pf = ParticleFilter(motion_model, observation_model)
responder = SimulatorResponder(pf)    
server = WebsocketServer(4567, host='127.0.0.1')
server.set_fn_new_client(responder.on_connect)
server.set_fn_message_received(responder.on_message)
server.run_forever()

Listening on port 4567 for clients..
Client connected
